In [48]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from sklearn import model_selection
import numpy as np

In [49]:
data = pd.read_csv("ml-100k/u.data",sep="\t", header=None)
data1 = pd.read_csv("ml-100k/u.user",sep="|",header=None)
data.columns = ['user id', 'movie id', 'rating', 'timestamp']
data1.columns = ['user id','age','gender','occupation','zip code']
data['movie id'].unique
df1= pd.DataFrame(data1)
df = pd.DataFrame(data)
label_encoder = LabelEncoder()
data = pd.merge(df,df1[['user id','age','gender','occupation']],on='user id',how='left')

# data = merged
data['encoded age']=label_encoder.fit_transform(data['age'])
data['encoded gender']=label_encoder.fit_transform(data['gender'])
data['encoded occupation']=label_encoder.fit_transform(data['occupation'])
# print(data1['encoded age'][2])
# print(data1)
data.head()

,user id,movie id,rating,timestamp,age,gender,occupation,encoded age,encoded gender,encoded occupation
0,196,242,3,881250949,49,M,writer,39,1,20
1,186,302,3,891717742,39,F,executive,29,0,6
2,22,377,1,878887116,25,M,writer,15,1,20
3,244,51,2,880606923,28,M,technician,18,1,19
4,166,346,1,886397596,47,M,educator,37,1,3


In [50]:
movies = pd.read_csv("ml-100k/u.item",
                    sep="|", encoding='latin-1', header=None)
movies.columns = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL',
                 'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy',
                 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
                 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [51]:
map_id_movie = {}
for id,row in movies.iterrows():
  map_id_movie[row['movie id']] = row['movie title']
map_id_movie[1398]

'Anna (1996)'

In [52]:
num_users = data['user id'].nunique()
# nusers = data1['user id'].nunique()
num_age = data['encoded age'].nunique()
num_occupation = data['encoded occupation'].nunique()
num_gender = data['encoded gender'].nunique()
num_movies = data['movie id'].nunique()
print(
    (f"Number of users: {num_users}\n"
    f"Number of movies: {num_movies}\n"
    f"Number of ages: {num_age}\n"
    f"Number of occupations: {num_occupation}\n"
    f"Number of genders: {num_gender}")
)

Number of users: 943
Number of movies: 1682
Number of ages: 61
Number of occupations: 21
Number of genders: 2


In [53]:
data_train,data_test = model_selection.train_test_split(data, test_size = 0.1,random_state = 42,stratify = data['rating'])
data_test

,user id,movie id,rating,timestamp,age,gender,occupation,encoded age,encoded gender,encoded occupation
24188,299,88,3,889502902,29,M,doctor,19,1,2
14023,347,462,2,881654359,18,M,student,8,1,18
20170,96,185,5,884403866,25,F,artist,15,0,1
87853,880,302,5,880166451,13,M,student,3,1,18
8174,177,289,2,880130534,20,M,programmer,10,1,14
...,...,...,...,...,...,...,...,...,...,...
18733,43,215,5,883955467,29,F,librarian,19,0,10
83494,860,516,3,885991040,70,F,retired,59,0,15
36379,313,484,5,891016193,41,M,marketing,31,1,11
17294,112,286,4,884992484,30,M,salesman,20,1,16


In [54]:
class MF(nn.Module):
  def __init__(self,users,movie_ids,ages,occupations,genders,embed_size):
    super(MF,self).__init__()
    self.user_embed = nn.Embedding(users,int(embed_size/4))
    self.user_bias = nn.Embedding(users, 1)
    self.age_embed = nn.Embedding(ages,int(embed_size/4))
    self.age_bias = nn.Embedding(ages,1)
    self.occupation_embed = nn.Embedding(occupations,int(embed_size/4))
    self.occupation_bias = nn.Embedding(occupations,1)
    self.gender_embed = nn.Embedding(genders,int(embed_size/4))
    self.gender_bias = nn.Embedding(genders,1)
    self.movie_embed = nn.Embedding(movie_ids,embed_size)
    self.movie_bias = nn.Embedding(movie_ids,1)
    self.user_embed.weight.data.uniform_(0,0.05)
    self.age_embed.weight.data.uniform_(0,0.05)
    self.occupation_embed.weight.data.uniform_(0,0.05)
    self.gender_embed.weight.data.uniform_(0,0.05)
    self.movie_embed.weight.data.uniform_(0,0.05)
    self.user_bias.weight.data.uniform_(-0.01,0.01)
    self.age_bias.weight.data.uniform_(-0.01,0.01)
    self.occupation_bias.weight.data.uniform_(-0.01,0.01)
    self.gender_bias.weight.data.uniform_(-0.01,0.01)
    self.movie_bias.weight.data.uniform_(-0.01,0.01)


  def forward(self,u,v,a,o,g):
    # print(u.shape)
    # print(v.shape)
    U = self.user_embed(u)
    # print(U.shape)
    V = self.movie_embed(v)
    # print(V.shape)
    A = self.age_embed(a)
    O = self.occupation_embed(o)
    G = self.gender_embed(g)
    b_u = self.user_bias(u).squeeze()
    b_v = self.movie_bias(v).squeeze()
    b_a = self.age_bias(a).squeeze()
    b_o = self.occupation_bias(o).squeeze()
    b_g = self.gender_bias(g).squeeze()
    user_vec = torch.cat([U,A,O,G], dim=-1)
    return (user_vec*V).sum(1) + b_u + b_v + b_a + b_o + b_g


In [55]:
def train_epochs(model,lr,epochs,Nueralnet = False):
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  loss_fn = nn.MSELoss()
  for t in range(epochs):
    model.train()
    users = torch.LongTensor(data_train['user id'].to_numpy()) # .cuda()
    items = torch.LongTensor(data_train['movie id'].to_numpy()) #.cuda()
    ages = torch.LongTensor(data_train['encoded age'].to_numpy())
    occupations = torch.LongTensor(data_train['encoded occupation'].to_numpy())
    genders = torch.LongTensor(data_train['encoded gender'].to_numpy())
    ratings = torch.FloatTensor(data_train['rating'].to_numpy())
    if Nueralnet:
        ratings = ratings.unsqueeze(1)
        y_pred = model(users,items)
    else:
        y_pred = model(users,items,ages,occupations,genders)
 #.cuda()
    # y_pred = model(users,items,ages,occupations,genders)
    loss_train = loss_fn(y_pred,ratings)

    # Backpropagation
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        test_users = torch.LongTensor(data_test['user id'].to_numpy())
        test_items = torch.LongTensor(data_test['movie id'].to_numpy())
        test_ages = torch.LongTensor(data_test['encoded age'].to_numpy())
        test_occupations = torch.LongTensor(data_test['encoded occupation'].to_numpy())
        test_genders = torch.LongTensor(data_test['encoded gender'].to_numpy())
        test_ratings = torch.FloatTensor(data_test['rating'].to_numpy())
        if Nueralnet:
            test_ratings=test_ratings.unsqueeze(1)
            pred_test = model(test_users,test_items)
        else:
            pred_test = model(test_users,test_items,test_ages,test_occupations,test_genders)
        loss_test = loss_fn(pred_test, test_ratings)

    print(f"Epoch {t+1} -- Train loss: {loss_train:>7f} Test loss: {loss_test:>7f}")


In [56]:
model = MF(num_users+1, num_movies+1,num_age+1,num_occupation+1,num_gender+1,embed_size=60)
train_epochs(model,0.04,60)
print("Model trained")

Epoch 1 -- Train loss: 13.476962 Test loss: 10.746129
Epoch 2 -- Train loss: 10.746518 Test loss: 7.512383
Epoch 3 -- Train loss: 7.510556 Test loss: 4.300903
Epoch 4 -- Train loss: 4.297201 Test loss: 1.906981
Epoch 5 -- Train loss: 1.900885 Test loss: 1.292031
Epoch 6 -- Train loss: 1.274644 Test loss: 2.538216
Epoch 7 -- Train loss: 2.489333 Test loss: 3.593486
Epoch 8 -- Train loss: 3.512446 Test loss: 3.341568
Epoch 9 -- Train loss: 3.243386 Test loss: 2.389913
Epoch 10 -- Train loss: 2.288691 Test loss: 1.499593
Epoch 11 -- Train loss: 1.404516 Test loss: 1.044209
Epoch 12 -- Train loss: 0.959463 Test loss: 1.024415
Epoch 13 -- Train loss: 0.950346 Test loss: 1.253132
Epoch 14 -- Train loss: 1.187907 Test loss: 1.532031
Epoch 15 -- Train loss: 1.473022 Test loss: 1.733392
Epoch 16 -- Train loss: 1.677929 Test loss: 1.804318
Epoch 17 -- Train loss: 1.749957 Test loss: 1.743884
Epoch 18 -- Train loss: 1.688439 Test loss: 1.582169
Epoch 19 -- Train loss: 1.523695 Test loss: 1.366928

In [57]:
class NueralNet(nn.Module):
  def __init__(self,users,movie_ids,embed_size):
    super(NueralNet,self).__init__()
    self.user_embed = nn.Embedding(users,embed_size)
    self.movie_embed = nn.Embedding(movie_ids,embed_size)
    self.user_embed.weight.data.uniform_(0,0.05)
    self.movie_embed.weight.data.uniform_(0,0.05)
    self.network = nn.Sequential(
            nn.Linear(2*embed_size,64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    self.output = nn.Sigmoid()

  def forward(self,u,v):
    U = self.user_embed(u)
    V = self.movie_embed(v)
    vector = torch.cat([U,V], dim=-1)
    out_temp = self.network(vector)
    return self.output(out_temp)*5

In [58]:
model_2 = NueralNet(num_users+1,num_movies+1,64)
train_epochs(model_2,0.001,170,True)

Epoch 1 -- Train loss: 2.527740 Test loss: 2.507633
Epoch 2 -- Train loss: 2.507829 Test loss: 2.487837
Epoch 3 -- Train loss: 2.488024 Test loss: 2.468355
Epoch 4 -- Train loss: 2.468524 Test loss: 2.449847
Epoch 5 -- Train loss: 2.450004 Test loss: 2.431785
Epoch 6 -- Train loss: 2.431925 Test loss: 2.413538
Epoch 7 -- Train loss: 2.413655 Test loss: 2.395025
Epoch 8 -- Train loss: 2.395121 Test loss: 2.376491
Epoch 9 -- Train loss: 2.376558 Test loss: 2.359391
Epoch 10 -- Train loss: 2.359435 Test loss: 2.342563
Epoch 11 -- Train loss: 2.342593 Test loss: 2.325254
Epoch 12 -- Train loss: 2.325260 Test loss: 2.307421
Epoch 13 -- Train loss: 2.307398 Test loss: 2.289018
Epoch 14 -- Train loss: 2.288962 Test loss: 2.269980
Epoch 15 -- Train loss: 2.269884 Test loss: 2.250237
Epoch 16 -- Train loss: 2.250093 Test loss: 2.229740
Epoch 17 -- Train loss: 2.229543 Test loss: 2.208473
Epoch 18 -- Train loss: 2.208212 Test loss: 2.186997
Epoch 19 -- Train loss: 2.186671 Test loss: 2.166260
Ep

In [59]:
user_id = input("Enter the user_id for movie recommendations: ")
movie_list = data[data['user id'] == int(user_id)]['movie id'].to_list()
unseen_list = [int(x) for x in range(1,num_movies+1) if x not in movie_list]
unseen_list = np.array(unseen_list)
temp_x = torch.LongTensor(np.full(np.size(unseen_list),int(user_id)))
temp_age = torch.LongTensor(np.full(np.size(unseen_list),int(data['encoded age'][int(user_id)-1])))
temp_occ = torch.LongTensor(np.full(np.size(unseen_list),int(data['encoded occupation'][int(user_id)-1])))
temp_gen = torch.LongTensor(np.full(np.size(unseen_list),int(data['encoded gender'][int(user_id)-1])))
unseen_mov = torch.LongTensor(unseen_list)


In [60]:
model.eval()
predictions_MF = model(temp_x,unseen_mov,temp_age,temp_occ,temp_gen)
sorted_indices = np.argsort(predictions_MF.detach().numpy())[::-1]
ordered_movies = unseen_list[sorted_indices]
ordered_ratings = predictions_MF.detach().numpy()[sorted_indices]
print("-------------Recommended movies by MF--------------")
i=0
for xyz in ordered_movies[:10]:
  print(map_id_movie[xyz],ordered_ratings[i])
  i=i+1

-------------Recommended movies by MF--------------
American Dream (1990) 9.839307
Grosse Fatigue (1994) 8.996676
Enfer, L' (1994) 8.4799
Two or Three Things I Know About Her (1966) 8.185433
Safe Passage (1994) 8.14035
Ladybird Ladybird (1994) 7.870578
Delta of Venus (1994) 7.8434196
World of Apu, The (Apur Sansar) (1959) 7.757298
Little City (1998) 7.6582255
Legal Deceit (1997) 7.634438


In [61]:
model_2.eval()
predictions_NN = model_2(temp_x,unseen_mov)
predictions_NN=predictions_NN.squeeze()
sorted_indices = np.argsort(predictions_NN.detach().numpy())[::-1]
ordered_movies = unseen_list[sorted_indices]
ordered_ratings = predictions_NN.detach().numpy()[sorted_indices]
print("---------------Recommended movies by NeuralNetworks--------------")
i=0
for xyz in ordered_movies[:10]:
  print(map_id_movie[xyz],ordered_ratings[i])
  i=i+1

---------------Recommended movies by NeuralNetworks--------------
Safe Passage (1994) 4.869988
Great Day in Harlem, A (1994) 4.848739
Someone Else's America (1995) 4.8365054
Santa with Muscles (1996) 4.8309402
Entertaining Angels: The Dorothy Day Story (1996) 4.8307023
Pather Panchali (1955) 4.829671
Saint of Fort Washington, The (1993) 4.8254547
The Deadly Cure (1996) 4.8253293
Some Mother's Son (1996) 4.8247833
Bitter Sugar (Azucar Amargo) (1996) 4.814805
